In [1]:
###############
## Libraries ##
###############

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

# Training the Model

In [7]:
##########################
## Load in MNIST Digits ##
##########################

all_data = np.load("/scratch/gpfs/eysu/src_data/mnist.npz")
print(all_data.files)
x_test = all_data['x_test']
x_train = all_data['x_train']
y_train = all_data['y_train']
y_test = all_data['y_test']

print(x_test.shape)
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

['x_test', 'x_train', 'y_train', 'y_test']
(10000, 28, 28)
(60000, 28, 28)
(60000,)
(10000,)


In [8]:
###############################
## Partition and resize data ##
###############################

labels = ["0",  # index 0
          "1",  # index 1
          "2",  # index 2 
          "3",  # index 3 
          "4",  # index 4
          "5",  # index 5
          "6",  # index 6 
          "7",  # index 7 
          "8",  # index 8 
          "9"]  # index 9

# save train labels
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train_labels = y_train
y_test_labels = y_test

# Further break training data into train / validation sets (# put 5000 into validation set and keep remaining 55,000 for train)
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

# Validation set
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [10]:
###################################################
## Shuffle y_train such that the training images ##
## and labels no longer match up                 ##
###################################################

y_train_shuffle = np.copy(y_train)
np.random.shuffle(y_train_shuffle)
print(y_train)
print(y_train_shuffle)

[7 3 4 ... 5 6 8]
[3 9 2 ... 7 6 1]


In [12]:
##############################################################
## This cell runs the iterated learning training procedure. ##
##############################################################

# Number of iterations in the serial reproduction
MAX_ITER = 25
# Number of epochs per training run
EPOCHS = 10
# Where to store output model weights and softmax predictions
save_path = "/scratch/gpfs/eysu/mock_supervised_weights/"

for iteration in range(0,MAX_ITER):
    # If iteration is seed, train on original target vectors, else, train on y_hat from time t-1
    if iteration == 0:
        # One-hot encode the labels
        # Pass the randomized labels to the model as y_train
        y_train = tf.keras.utils.to_categorical(y_train_shuffle, 10)
        mpth = 'model.weights.best.hdf5'
        y_hat_test_name = 'y_hat_test_seed'
        y_hat_train_name = 'y_hat_train_seed'      
    elif iteration > 0:
        # Key step: set new targets as y_hat
        y_train = y_hat    
        mpth = 'model.weights.best.' + 'iter' + str(iteration) + '.hdf5'
        y_hat_test_name = 'y_hat_test_' + 'iter' + str(iteration)
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(iteration)

    # Define the model: a small CNN model (could probably be done outside loop)
    model = tf.keras.Sequential()

    # Must define the input shape in the first layer of the neural network
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # Take a look at the model summary
    # model.summary()

    # define optimization and energy parameters
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Save checkpoints
    checkpointer = ModelCheckpoint(filepath= save_path + mpth, verbose = 1, save_best_only=False) #True
    # Train the model
    model.fit(x_train,
             y_train,
             batch_size=64,
             epochs=EPOCHS,
             validation_data=(x_valid, y_valid),
             callbacks=[checkpointer])

    # Load the weights with the best validation accuracy
    y_hat = model.predict(x_train) #feed back serial reproduction targets
    y_hat_test = model.predict(x_test)
    
    model.load_weights(save_path + mpth)
    # Evaluate the model on test set
    score = model.evaluate(x_test, y_test, verbose=0)
    # Print test accuracy
    print('\n', 'Test accuracy:', score[1])

    # Save results for each iteration in the serial reproduction chain
    np.save(save_path + y_hat_train_name + '.npy', y_train)
    print(save_path + y_hat_train_name)

    np.save(save_path + y_hat_test_name + '.npy', y_hat_test)
    print(save_path + y_hat_test_name)



2022-10-13 11:51:11.080481: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 11:51:14.484378: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78971 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:ca:00.0, compute capability: 8.0


Epoch 1/10


2022-10-13 11:51:20.389775: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401
2022-10-13 11:51:24.579905: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-13 11:51:24.736039: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


860/860 [==============================] - ETA: 0s - loss: 2.3025 - accuracy: 0.1086
Epoch 1: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.hdf5
860/860 [==============================] - 12s 2ms/step - loss: 2.3025 - accuracy: 0.1086 - val_loss: 2.3013 - val_accuracy: 0.1126
Epoch 2/10
849/860 [============================>.] - ETA: 0s - loss: 2.3017 - accuracy: 0.1121
Epoch 2: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3017 - accuracy: 0.1120 - val_loss: 2.3009 - val_accuracy: 0.1126
Epoch 3/10
852/860 [============================>.] - ETA: 0s - loss: 2.3015 - accuracy: 0.1109
Epoch 3: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3015 - accuracy: 0.1112 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 4/10
849/860 [============================>.] -

860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 6/10
835/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 6: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter2.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 7/10
833/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 7: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter2.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 8/10
839/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 8: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter2.hdf5
860/860 [================

...

837/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 6: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter14.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 7/10
842/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 7: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter14.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 8/10
843/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 8: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter14.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 9/10
842/860 [=============

851/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 10: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter16.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126

 Test accuracy: 0.11349999904632568
/scratch/gpfs/eysu/mock_supervised_weights/y_hat_train_iter16
/scratch/gpfs/eysu/mock_supervised_weights/y_hat_test_iter16
Epoch 1/10
839/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 0.9922
Epoch 1: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter17.hdf5
860/860 [==============================] - 2s 2ms/step - loss: 2.3009 - accuracy: 0.9924 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 2/10
840/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 2: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter17.hdf5
860

Epoch 4/10
837/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 4: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter19.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 5/10
837/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 5: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter19.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 6/10
837/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 6: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter19.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 7/10
837/860 [==

840/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 8: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter21.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 9/10
833/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 9: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter21.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 10/10
836/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 10: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter21.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126

 Test accuracy: 0.113499999046

Epoch 2/10
843/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 2: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter24.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 3/10
845/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 3: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter24.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 4/10
844/860 [============================>.] - ETA: 0s - loss: 2.3009 - accuracy: 1.0000
Epoch 4: saving model to /scratch/gpfs/eysu/mock_supervised_weights/model.weights.best.iter24.hdf5
860/860 [==============================] - 1s 2ms/step - loss: 2.3009 - accuracy: 1.0000 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 5/10
844/860 [==

# Ideas
- visualize the softmax matrices of 200 random images. One hot seed vectors should be random. Does the model perform better at categorizing each time? How will things converge?
- quantify the final categories predicted. What is the distribution between the 10 classes?
    - interesting thing to try might be line graphs showing the fraction of images in each class taken every 5 iterations. Hopefully will see changes in the lines over time?
- What if we don't provide an input # classes? This is done thorugh the .to_categorical step. Look into if there is a way to let this be determined naturally

# Initial Visualization

In [13]:
#######################################################
## Look at softmax output matrices for random images ##
#######################################################

y_hat_train_arr = np.zeros([y_train.shape[0], len(labels), MAX_ITER])
for i in range(MAX_ITER):
    if i == 0:
        y_hat_train_name = 'y_hat_train_seed'
       
    else:
        y_hat_train_name = 'y_hat_train_' + 'iter' + str(i)
        
    # Load test set softmax outputs 
    yhtr = np.load(save_path + y_hat_train_name + '.npy')

    # The first time through, use binary weight vectors to save seed array
    # Recall that these initial labels were randomized and do not correlate to 
    # the image's given class in the dataset
    
    if i == 0:
        true_class_tr = np.nonzero(yhtr)[1]   
    y_hat_train_arr[:, :, i] = yhtr

print(y_hat_train_arr.shape)
# (55000, 10, 25)

(55000, 10, 25)


In [15]:
########################
## Visualize N images ##
########################

from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')
import matplotlib.backends.backend_pdf

def visualize_softmax(image_idx):

    # visualize each image
    figure = plt.figure(figsize=(40, 40))
    # plot image
    ax1 = figure.add_subplot(8, 8, 1, xticks=[], yticks=[])
    im1 = ax1.imshow(x_train[image_idx])
    ax1.set_title("Image")

    # plot weights graph
    ax2 = figure.add_subplot(8, 8, 2)
    im2 = ax2.imshow(y_train_arr[image_idx, :, :].T, cmap='Wistia')

    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = figure.colorbar(im2, cax=cax, orientation='vertical', ticks=[0, 1])
    cbar.ax.set_yticklabels(['0', '1'])

    ax2.set(xlabel='Classes', ylabel='Iterations', title='Softmax Outputs')
    ax2.set_xticks(ticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    ax2.set_xticklabels(labels)

    pdf.savefig(figure, bbox_inches='tight')
    plt.show()
        
save_image_path = "home/eysu/COS454-Iterated-Learning-CNN-bias/Mock_Supervised_Outputs/"
pdf = matplotlib.backends.backend_pdf.PdfPages(save_image_path + "sample_200_mock_supervised_softmax_outputs.pdf")
for i in range(200):
    visualize_softmax(np.random_integers(0, y_train_arr.shape[0]))
pdf.close()

FileNotFoundError: [Errno 2] No such file or directory: 'home/eysu/COS454-Iterated-Learning-CNN-bias/Mock_Supervised_Outputs/sample_200_mock_supervised_softmax_outputs.pdf'